In [19]:
import ffmpeg
import os

import numpy as np

out = ffmpeg.probe('/Users/sean/mbp_storage/240420/fwdFacing/GX010037.MP4',)

In [4]:
out.keys()

dict_keys(['streams', 'format'])

In [17]:
out['streams'][0]['duration']

'2114.112000'

In [16]:
out['streams'][0]['tags']['creation_time']

'2024-04-20T17:09:28.000000Z'

In [72]:
files = os.listdir('/Users/sean/mbp_storage/timeStamp/')

In [73]:
files

['GX050037.MP4',
 'GX070046.MP4',
 'GX040037.MP4',
 'GX060046.MP4',
 'GX040046.MP4',
 'GX060037.MP4',
 'GX050046.MP4',
 'GX070037.MP4',
 'GX020037.MP4',
 'GX010046.MP4',
 'GX030037.MP4',
 'GX080046.MP4',
 'GX010037.MP4',
 'GX030046.MP4',
 'GX080037.MP4',
 'GX020046.MP4']

In [113]:
videos = []
chapters = []
prefix = []
suffix = []
s=''
for file in files:
    videos.append(s.join(list(file)[4:8]))
    chapters.append(s.join(list(file)[2:4]))
    prefix.append(s.join(list(file)[:2]))
    suffix.append(s.join(list(file)[8:]))
    

In [114]:
import pandas as pd

In [115]:
df = pd.DataFrame({'video': videos, 'chapter': chapters, 'prefix': prefix, 'suffix':suffix})

In [116]:
df = pd.DataFrame({'video': videos, 'chapter': chapters, 'prefix': prefix, 'suffix':suffix})
df.sort_values(by=['video', 'chapter'],inplace=True, ignore_index=True)

In [120]:
files = []
currentVideo = []
firstChapters_ind = []
otherChapters_ind = []
for i in range(len(df)):
    if currentVideo != str(df['video'][i]):
        currentVideo = str(df['video'][i])
        firstChapters_ind.append(i)
    else:
        otherChapters_ind.append(i)
    files.append(str(df['prefix'][i])+str(df['chapter'][i])+str(df['video'][i])+str(df['suffix'][i]))

In [121]:
files

['GX010037.MP4',
 'GX020037.MP4',
 'GX030037.MP4',
 'GX040037.MP4',
 'GX050037.MP4',
 'GX060037.MP4',
 'GX070037.MP4',
 'GX080037.MP4',
 'GX010046.MP4',
 'GX020046.MP4',
 'GX030046.MP4',
 'GX040046.MP4',
 'GX050046.MP4',
 'GX060046.MP4',
 'GX070046.MP4',
 'GX080046.MP4']

In [122]:
firstChapters_ind

[0, 8]

In [123]:
otherChapters_ind

[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]

In [174]:
import datetime
timeStamps = []
timeDeltas = []
for i in range(len(files)):
    metaData = ffmpeg.probe(os.path.join('/Users/sean/mbp_storage/timeStamp/',files[i]))
    if i in firstChapters_ind:
        origin = datetime.datetime.strptime(metaData['streams'][0]['tags']['creation_time'], '%Y-%m-%dT%H:%M:%S.%fZ')
        timeStamp = datetime.datetime.strptime(metaData['streams'][0]['tags']['creation_time'], '%Y-%m-%dT%H:%M:%S.%fZ')
        duration =  datetime.timedelta(seconds=float(metaData['streams'][0]['duration']))
        timeSince = datetime.timedelta(seconds=float(metaData['streams'][0]['duration']))
        timeDeltas.append(datetime.timedelta(seconds=0))
        timeStamps.append(timeStamp)
        
        
    else:
        timeStamp = origin + timeSince
        duration =  datetime.timedelta(seconds=float(metaData['streams'][0]['duration']))
        timeDeltas.append(timeSince)
        timeStamps.append(timeStamp)
        timeSince = timeSince + duration

        
        
        

'2024-04-20 17:09:28'

'0:35:14.112000'

'2114.112000'

In [177]:
import csv

In [1]:

def Write_CSV():
    with open(os.path.join('/Users/sean/mbp_storage/timeStamp/','timeStamping.csv'), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['File', 'TimeStamp', 'NjordOffset'])
        for i in range(len(timeStamps)):

            writer.writerow([files[i], str(timeStamps[i]), str(timeDeltas[i].seconds)+'.'+str(timeDeltas[i].microseconds) ])
    
    csvfile.close()

In [2]:
import veeringVideo

veeringVideo.TimeStamping('/Users/sean/mbp_storage/timeStamp/').Add_CSV()